# Genetic Tuner Tutorial

This notebook demonstrates how to use the `GeneticTuner` to find optimal hyperparameters for a TimeSeries Agent.

In [ ]:
import os
import pandas as pd
import numpy as np
from collections import Counter
from timeseries_agent.tuners.genetic import GeneticTuner
from timeseries_agent.api import load_agent
from timeseries_agent.utils.extras import plot_evolution_of_fitness_scores_across_generations, plot_average_diversity_scores_across_generations
import warnings
warnings.filterwarnings("ignore")

## 1. Get Synthetic Data

In [ ]:
# Load data
df = pd.read_csv("https://raw.githubusercontent.com/datasets/global-temp/main/data/monthly.csv")
df = df[df['Source'] == 'gcag']
df = df[['Year', 'Mean']]
df['temperature'] = df['Mean']
df.drop(columns=['Mean'], inplace=True)

# Train test split
target_column = 'temperature'
df = df.head(1000)
last_n_test_rows = 100
train_data = df.iloc[:-last_n_test_rows]
test_data = df.tail(last_n_test_rows)

# Add lags
train_data['Lag1'] = train_data[target_column].shift(1)
train_data['Lag2'] = train_data[target_column].shift(2)
test_data['Lag1'] = test_data[target_column].shift(1)
test_data['Lag2'] = test_data[target_column].shift(2)

train_data = train_data.dropna(subset=['Lag1', 'Lag2'])
test_data = test_data.dropna(subset=['Lag1', 'Lag2'])

# Save data to dir
DATA_DIR = 'data'
os.makedirs(DATA_DIR, exist_ok=True)
train_csv_path = os.path.join(DATA_DIR, 'global_temp_train.csv')
test_csv_path = os.path.join(DATA_DIR, 'global_temp_test.csv')
train_data.to_csv(train_csv_path, index=False)
test_data.to_csv(test_csv_path, index=False)

## 2. Define Parameter Search Space

In [ ]:
# Parameter grid to explore
params_grid = {
    'learning_rate': [0.001, 0.0005, 0.0001],
    'lookback': [5, 10, 15],
    'hidden_layers': [
        [50, 50],
        [100, 100, 10],
        [200, 100, 50],
    ],
}

# Base parameters that will be the same for all models
base_params = {
    'csv_path': train_csv_path,
    'feature_cols': ['Lag1', 'Lag2', 'temperature'],
    'target_col': 'temperature',
    'env_kwargs': {
        'normalize_state': True,
        'test_size': 0.2
    },
    'agent_kwargs': {
        'agent_type': 'reinforce',
        'output_size': 3,
    },
    'trainer_kwargs': {
        'max_epochs': 10,
        'enable_checkpointing': True,
    }
}

# Genetic Algorithm Configuration
genetic_params = {
    'population_size': 5,               # Size of each generation's population
    'num_generations': 3,               # Number of generations to evolve
    'mutation_rate': 0.1,               # Probability of parameter mutation
    'elitism_count': 1,                 # Number of best individuals to preserve
    'initial_temperature': 100.0,       # Initial temperature for simulated annealing
    'cooling_rate': 0.95,               # Rate at which temperature decreases
}

## 3. Run Genetic Tuner

In [ ]:
# Create genetic tuner instance
tuner = GeneticTuner(
    base_log_dir="logs/genetic_tuning",
    **genetic_params
)

# Train models using genetic algorithm optimization
results = tuner.train(
    params_grid=params_grid,
    base_params=base_params,
)

## 4. Analyze Results

In [ ]:
display(results.head())

best_model = results.iloc[0]
print("Best Model Configuration:")
for param, value in best_model.items():
    print(f"{param}: {value}")

### 4.a. Visualize Evolution

In [ ]:
plot_evolution_of_fitness_scores_across_generations(results)
plot_average_diversity_scores_across_generations(results)

## 5. Evaluate Best Model

In [ ]:
best_model_params = tuner.best_model_params
best_model_checkpoint = tuner.best_model_checkpoint

env_kwargs = base_params['env_kwargs']
env_kwargs['lookback'] = best_model_params['lookback']

loaded_agent = load_agent(
    checkpoint_path=best_model_checkpoint,
    csv_path=test_csv_path,
    feature_cols=base_params['feature_cols'],
    target_col=base_params['target_col'],
    agent_type='reinforce_step',
    **env_kwargs
)

def get_true_action(current_val, next_val):
    true_action = 0 if next_val > current_val else 1 if next_val < current_val else 2
    return true_action

def get_batch_predictions(df, lookback_size):
    predictions = []
    true_actions = []
    for i in range(lookback_size, len(df)):
        if i >= len(df) - 1:
            continue
        current_features = df[base_params['feature_cols']].iloc[i-lookback_size:i].values.astype(np.float32)
        current_target = df[base_params['target_col']].iloc[i]
        pred_action, probs = loaded_agent.act(current_features, return_probs=True)
        predictions.append(pred_action)
        next_target = df[base_params['target_col']].iloc[i+1]
        true_action = get_true_action(current_target, next_target)
        true_actions.append(true_action)
    return true_actions, predictions

df_test = pd.read_csv(test_csv_path)
y_true, y_pred = get_batch_predictions(df_test, best_model_params['lookback'])
print(f'\n true dist == {Counter(y_true)}, pred dist == {Counter(y_pred)}')